<a href="https://colab.research.google.com/github/theocharistr/Natural_Language_Processing/blob/main/Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets



In [2]:
import datasets
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification,TrainingArguments

In [3]:
# Load the MultiNERD dataset

from datasets import load_dataset

dataset = load_dataset("Babelscape/multinerd")

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [4]:
# Filter out non-English examples

english_dataset = dataset.filter(lambda x: x["lang"] == "en")

In [5]:
english_dataset['train'][0]

{'tokens': ['The', 'type', 'locality', 'is', 'Kīlauea', '.'],
 'ner_tags': [0, 0, 0, 0, 5, 0],
 'lang': 'en'}

In [6]:
english_dataset

DatasetDict({
    train: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 262560
    })
    validation: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32820
    })
    test: Dataset({
        features: ['tokens', 'ner_tags', 'lang'],
        num_rows: 32908
    })
})

In [7]:
# Tokenize and encode the text
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")


In [8]:
def tokenize_function(examples):
    # Adjust max_length to the desired value (e.g., 560 or any other suitable value)
    max_length = 560

    # Tokenize each sublist and concatenate the results
    encoded_tokens = [tokenizer(sublist, truncation=True, padding='max_length', max_length=max_length, return_tensors='pt')['input_ids'].tolist()[0] for sublist in examples["tokens"]]

    # Ensure each element is a list and convert non-list values to lists
    padded_tokens = [tokens if isinstance(tokens, list) else [tokens] for tokens in encoded_tokens]

    # Create a dictionary to store the encoded tokens
    encoded_data = {"input_ids": padded_tokens}

    return encoded_data


In [9]:
english_dataset_encoded = english_dataset.map(tokenize_function, batched=True)


Map:   0%|          | 0/262560 [00:00<?, ? examples/s]

Map:   0%|          | 0/32820 [00:00<?, ? examples/s]

Map:   0%|          | 0/32908 [00:00<?, ? examples/s]

Filter:   0%|          | 0/262560 [00:00<?, ? examples/s]

Filter:   0%|          | 0/32820 [00:00<?, ? examples/s]

Filter:   0%|          | 0/32908 [00:00<?, ? examples/s]

In [11]:
# Fine-tune the chosen model on the English subset of the training set (System A)
model = AutoModelForTokenClassification.from_pretrained("bert-base-uncased", num_labels=7)

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [21]:
training_args = transformers.TrainingArguments(
    output_dir="./outputs",
    num_train_epochs=3,
    per_device_train_batch_size=16,
)

trainer = transformers.Trainer(
    model=model,
    args=training_args,
    train_dataset=filtered_dataset,
)


# Print the length of the training dataset
print("Number of training samples:", len(filtered_dataset))


Number of training samples: 3


In [46]:
from datasets import load_dataset
import pandas as pd

# Load the dataset
dataset = load_dataset("Babelscape/multinerd", split="train")

# Define the entity types you want to keep
entity_types = ["PER", "ORG", "LOC"]

# Create a filtered dataset for training
filtered_dataset = [x for x in dataset if any(tag in entity_types for tag in x["ner_tags"])]

# Convert the filtered dataset to a list of dictionaries
train_dataset_dicts = filtered_dataset

# Create a Pandas DataFrame from the list of dictionaries
train_dataset_df = pd.DataFrame(train_dataset_dicts)

# Save the DataFrame to a Parquet file
train_dataset_df.to_parquet("train_dataset.parquet")


Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/20 [00:00<?, ?it/s]

In [55]:

# Save the DataFrame to a Parquet file
train_dataset_df.to_parquet("train_dataset.parquet")

# Load the checkpoint file
trainer.train("train_dataset.parquet")

# Resume training from the checkpoint file
trainer.train("trained_model.ckpt")

ValueError: ignored

In [57]:
 trainer.train()

ValueError: ignored

In [ ]:
# Evaluate the model
eval_results = trainer.evaluate()
print(f"Evaluation results: {eval_results}")